In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications import inception_v3 as net
from keras import backend as K
from PIL import Image
import copy

In [ ]:
img = image.load_img("data/fork.png", target_size = (299, 299))

In [ ]:
img.show()

In [ ]:
input_image = image.img_to_array(img)
input_image /= 255
input_image -= 0.5
input_image *= 2.
input_image = np.expand_dims(input_image, axis=0)
hacked_image = copy.deepcopy(input_image)

In [ ]:
model = net.InceptionV3() # Changer ici!!
pred = model.predict(input_image)
pred_classes = net.decode_predictions(pred, top=1)
imagenet_id, name, confidence = pred_classes[0][0]
print("This is obviously a {} (with {:.4}% confidence)".format(name, confidence * 100))

In [ ]:
model_input_layer = model.layers[0].input
model_output_layer = model.layers[-1].output

object_to_fake = 963 # Pizza
confidence_function = model_output_layer[0, object_to_fake]

gradient_function = K.gradients(confidence_function, model_input_layer)[0]

confidence_and_gradients = K.function([model_input_layer, K.learning_phase()], [confidence_function, gradient_function])

confidence = 0

lr = 0.1

In [ ]:
while confidence < 0.98:
    confidence, gradients = confidence_and_gradients([hacked_image, 0])
    
    hacked_image += lr * gradients
    
    # hacked_image = np.clip(hacked_image, input_image - 0.1, input_image + 0.1)
    #hacked_image = np.clip(hacked_image, -1.0, 1.0)
    print("Confidence at {:.4}%".format(confidence * 100))

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(hacked_image[0])

In [ ]:
pred = model.predict(hacked_image)
pred_classes = net.decode_predictions(pred, top=1)
imagenet_id, name, confidence = pred_classes[0][0]
print("This is obviously a {} (with {:.4}% confidence)".format(name, confidence * 100))